# Feature Comparison between Semtab Business Data and Github Dataset
This Notbook investigates if contextual or any other features are inherently different between the 2 datasets. Existence of a significant difference would suggest reconsideration in the fine tuning data used.

In [1]:
import pandas as pd
from scipy.stats import mannwhitneyu

## Load Semtab Business Features

In [15]:
semtab_business_features_path = "B:/Projects/Alligator-2/alligator/Training_Data/results/companies-correct-qids_with_cols.csv"

In [16]:
semtab_business_data_df = pd.read_csv(semtab_business_features_path)

In [17]:
semtab_business_features_df = semtab_business_data_df.groupby("group").first().reset_index()
semtab_business_features_df.drop(columns=["group", "tableName", "key", "target"], inplace=True)
semtab_business_features_df.sort_index(axis=1, inplace=True)

## Load Github Data

In [5]:
github_dataset_features_path = "B:/Projects/Alligator-2/alligator/Github_Data/github_features.csv"

In [6]:
github_dataset_data_df = pd.read_csv(github_dataset_features_path)
github_dataset_features_df = github_dataset_data_df.drop(columns=["delta", "omega", "key"])
github_dataset_features_df.sort_index(axis=1, inplace=True)

## Useful Functions

In [7]:
def preprocess_features(data_path):
    """
    Reads JSON data from the specified path, extracts features for each entity,
    and returns a DataFrame with features as columns and a custom key as the index.
    The key is formatted as '{table_name}-{idRow}-{idColumn}' and accommodates multiple tables.

    Parameters:
        data_path (str): Path to the JSON file containing entity data.

    Returns:
        pd.DataFrame: A DataFrame with features as columns and custom key as the index.
    """
    # Read the JSON file
    with open(data_path, 'r') as f:
        data = json.load(f)

    # Initialize a list to store each row's features
    rows = []

    # Iterate over the tables (assuming 'semanticAnnotations' contains multiple tables)
    annotations = data.get('semanticAnnotations', {}).get('cea', [])
    if isinstance(annotations, list):  # Support single list structure as well
        annotations = [annotations]

    for table_annotations in annotations:
        table_name = data.get("tableName", "unknown_table")

        for annotation in table_annotations:
            # Ensure there is an entity present
            if annotation['entity']:
                first_entity = annotation['entity'][0]
                id_row = annotation.get('idRow')
                id_column = annotation.get('idColumn')

                # Generate the key in the format "{table_name}-{idRow}-{idColumn}"
                key = f"{table_name}-{id_row}-{id_column}"

                # Extract features as a dictionary
                features = {feature['id']: feature['value'] for feature in first_entity.get('features', [])}
                features['key'] = key  # Include the custom key in the DataFrame

                # Append the features dictionary to rows
                rows.append(features)

    # Create DataFrame and set 'key' as index
    df = pd.DataFrame(rows).set_index('key')

    return df

def extract_distribution_statistics(features_df):
    """
    Extracts statistical parameters for each feature in the DataFrame for distribution analysis.

    Parameters:
        features_df (pd.DataFrame): A DataFrame containing feature columns.

    Returns:
        pd.DataFrame: A DataFrame with ordered statistical parameters (mean, std_dev, min, 25%, median, 75%, max) for each feature.
    """
    # Initialize a dictionary to store statistics
    statistics = {}

    # Loop through each feature column to calculate statistics
    for column in features_df.columns:
        # Skip non-numeric columns
        if not pd.api.types.is_numeric_dtype(features_df[column]):
            continue

        # Calculate statistical parameters
        stats = {
            'mean': features_df[column].mean(),
            'std_dev': features_df[column].std(),
            'min': features_df[column].min(),
            '25%': features_df[column].quantile(0.25),
            'median': features_df[column].median(),
            '75%': features_df[column].quantile(0.75),
            'max': features_df[column].max(),
        }

        # Add stats to the dictionary
        statistics[column] = stats

    # Convert the statistics dictionary to a DataFrame
    stats_df = pd.DataFrame(statistics).T  # Transpose to make features as rows

    # Reorder columns
    stats_df = stats_df[['mean', 'std_dev', 'min', '25%', 'median', '75%', 'max']]

    return stats_df

def extract_correct_features(file_path):
    """
    Extracts features of the correct candidate entity for each row based on `rows.ids` and `semanticAnnotations.cea`.

    Parameters:
        file_path (str): Path to the JSON file.

    Returns:
        pd.DataFrame: A DataFrame with extracted features for the correct entity in each row.
    """
    # Load JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)

    extracted_data = []  # To store extracted features

    # Iterate over each row to find the correct ID
    for row in data['rows']:
        row_id = row['idRow']

        # Find the first valid ID in 'ids', ignoring empty or "NIL" entries
        correct_id = next((entity_id for entity_id in row['ids'] if entity_id != "NIL"), None)

        # Skip to the next row if there is no valid correct ID
        if correct_id is None:
            continue

        # Search in `cea` for matching idRow with idColumn = 2
        correct_entity = None
        for annotation in data['semanticAnnotations']['cea']:
            if annotation['idRow'] == row_id and annotation['idColumn'] == 2:
                # Look for the entity with the correct ID
                for entity in annotation['entity']:
                    if entity['id'] == correct_id:
                        correct_entity = entity
                        break
            if correct_entity:
                break

        # If the correct entity is found, extract features
        if correct_entity:
            features = {feature['id']: feature['value'] for feature in correct_entity.get('features', [])}
            features['idRow'] = row_id
            features['correct_id'] = correct_id
            extracted_data.append(features)

    # Convert to DataFrame
    extracted_df = pd.DataFrame(extracted_data)
    return extracted_df

def calculate_feature_correlations(features_df, method='pearson', plot=True):
    """
    Calculates the correlation matrix for the features in the DataFrame and optionally
    displays a high-resolution heatmap with a consistent color scale from -1 to +1.

    Parameters:
        features_df (pd.DataFrame): A DataFrame containing feature columns.
        method (str): Correlation method ('pearson', 'spearman', 'kendall').
        plot (bool): Whether to display a heatmap of the correlation matrix.

    Returns:
        pd.DataFrame: A DataFrame representing the correlation matrix of the features.
    """
    # Select only numeric columns for correlation calculation
    numeric_features = features_df.select_dtypes(include=['number'])

    # Calculate the correlation matrix
    correlation_matrix = numeric_features.corr(method=method)

    # Plot the heatmap if plot is True
    if plot:
        plt.figure(figsize=(24, 20), dpi=150)  # Large figure size with high DPI
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', square=True,
                    cbar_kws={'shrink': 0.8}, fmt=".2f", linewidths=0.5,
                    vmin=-1, vmax=1)  # Set color scale to -1 to +1 for accuracy
        plt.title(f'{method.capitalize()} Correlation Matrix', fontsize=18)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.yticks(fontsize=10)
        plt.tight_layout()  # Adjust layout for better spacing
        plt.show()

    return correlation_matrix

def plot_feature_distributions(features_df):
    """
    Creates a large figure with histogram charts for each feature in a grid layout,
    and violin plots: one for features with values <= 1, and individual plots for features with values > 1.

    Parameters:
        features_df (pd.DataFrame): A DataFrame containing feature columns.
    """
    # Select only numeric columns for plotting
    numeric_features = features_df.select_dtypes(include=['number'])

    # Calculate grid size based on the number of features
    num_features = numeric_features.shape[1]
    grid_size = math.ceil(math.sqrt(num_features))  # Square layout

    # Set up the figure size and layout for histograms
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(20, 20), dpi=150)
    axes = axes.flatten()  # Flatten to iterate easily

    # Plot histograms for each feature
    for i, column in enumerate(numeric_features.columns):
        sns.histplot(numeric_features[column], kde=True, ax=axes[i])
        axes[i].set_title(column)
        axes[i].set_xlabel('')
        axes[i].set_ylabel('Frequency')

    # Turn off any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    # Adjust layout for the histograms
    plt.tight_layout()
    plt.show()

    # Separate features based on value ranges
    features_leq_one = numeric_features.loc[:, numeric_features.max() <= 1]
    features_gt_one = numeric_features.loc[:, numeric_features.max() > 1]

    # Plot the violin plot for features with values <= 1
    if not features_leq_one.empty:
        plt.figure(figsize=(20, 10), dpi=150)
        sns.violinplot(data=features_leq_one, inner='quartile', palette='coolwarm')
        plt.xticks(rotation=45, ha='right')
        plt.title('Distribution of Features with Values <= 1')
        plt.show()

    # Plot individual violin plots for features with values > 1
    for column in features_gt_one.columns:
        plt.figure(figsize=(8, 8), dpi=150)
        sns.violinplot(y=features_gt_one[column], inner='quartile', palette='coolwarm')
        plt.title(f'Distribution of Feature: {column} (Values > 1)')
        plt.show()

def normalize_features(features_df, feature_range=(0, 1)):
    """
    Normalizes all numeric features in the DataFrame to a specified range using Min-Max scaling.

    Parameters:
        features_df (pd.DataFrame): A DataFrame containing feature columns.
        feature_range (tuple): Desired range for the scaled data (default is (0, 1)).

    Returns:
        pd.DataFrame: A DataFrame with normalized features.
    """
    # Select only numeric columns
    numeric_features = features_df.select_dtypes(include=['number'])

    # Initialize the MinMaxScaler with the desired range
    scaler = MinMaxScaler(feature_range=feature_range)

    # Fit and transform the numeric features
    normalized_data = scaler.fit_transform(numeric_features)

    # Create a DataFrame from the normalized data, retaining original feature names
    normalized_df = pd.DataFrame(normalized_data, columns=numeric_features.columns, index=features_df.index)

    # Return DataFrame with normalized features
    return normalized_df

def plot_feature_pairwise_scatter(features_df):
    """
    Creates a large scatter plot matrix for each pair of features in the DataFrame.

    Parameters:
        features_df (pd.DataFrame): A DataFrame containing feature columns.
    """
    # Select only numeric columns for plotting
    numeric_features = features_df.select_dtypes(include=['number'])

    # Set up the pairplot with a larger figure size
    sns.pairplot(numeric_features, height=2, plot_kws={'s': 10})  # 's' controls marker size

    # Show the plot
    plt.suptitle("Pairwise Scatter Plot of Features", y=1.02)  # Adjust title position
    plt.show()

# Compare Feature Distribution

In [8]:
extract_distribution_statistics(semtab_business_features_df)

,mean,std_dev,min,25%,median,75%,max
ambiguity_mention,0.089438,0.204601,0.000,0.020,0.023,0.043,1.000
cpa_t1,0.675959,0.404836,0.000,0.250,0.900,1.000,1.000
cpa_t2,0.341781,0.400609,0.000,0.000,0.055,0.778,1.000
cpa_t3,0.110151,0.252059,0.000,0.000,0.000,0.021,1.000
cpa_t4,0.031438,0.130991,0.000,0.000,0.000,0.000,1.000
cpa_t5,0.009168,0.063011,0.000,0.000,0.000,0.000,1.000
cta_t1,0.914126,0.183264,0.000,0.909,1.000,1.000,1.000
cta_t2,0.191219,0.317063,0.000,0.000,0.000,0.286,1.000
cta_t3,0.052608,0.168843,0.000,0.000,0.000,0.000,1.000
cta_t4,0.014509,0.079560,0.000,0.000,0.000,0.000,1.000


In [9]:
extract_distribution_statistics(github_dataset_features_df)

,mean,std_dev,min,25%,median,75%,max
ambiguity_mention,0.128346,0.256206,0.000,0.00,0.022,0.093,1.000
cpa_t1,0.021254,0.046103,0.000,0.00,0.000,0.015,0.487
cpa_t2,0.003394,0.013246,0.000,0.00,0.000,0.000,0.348
cpa_t3,0.000374,0.002812,0.000,0.00,0.000,0.000,0.122
cpa_t4,0.000014,0.000381,0.000,0.00,0.000,0.000,0.036
cpa_t5,0.000003,0.000177,0.000,0.00,0.000,0.000,0.033
cta_t1,0.133393,0.109787,0.000,0.04,0.120,0.200,1.000
cta_t2,0.042836,0.076461,0.000,0.00,0.000,0.080,1.000
cta_t3,0.017576,0.036532,0.000,0.00,0.000,0.040,0.320
cta_t4,0.007010,0.020316,0.000,0.00,0.000,0.000,0.280


In [10]:
# Dictionary to store test results for each feature
results = {}

for feature in semtab_business_features_df.columns:
    # Extract the feature data from each dataset
    data1 = semtab_business_features_df[feature]
    data2 = github_dataset_features_df[feature]
    
    # Perform the Mann-Whitney U Test
    stat, p_value = mannwhitneyu(data1, data2, alternative='two-sided')
    
    # Store the result
    results[feature] = {'statistic': stat, 'p_value': p_value}

# Convert results to a DataFrame for easier interpretation
results_df = pd.DataFrame(results).T
results_df

,statistic,p_value
ambiguity_mention,1.371559e+10,0.000000e+00
cpa_t1,2.159104e+10,0.000000e+00
cpa_t2,1.830046e+10,0.000000e+00
cpa_t3,1.557449e+10,0.000000e+00
cpa_t4,1.412909e+10,0.000000e+00
cpa_t5,1.336183e+10,0.000000e+00
cta_t1,2.492165e+10,0.000000e+00
cta_t2,1.332075e+10,1.283536e-191
cta_t3,1.150747e+10,0.000000e+00
cta_t4,1.171178e+10,0.000000e+00
